# Computation Intelligence for Optimization | Sports League Optimization

`Group AM`
- Eduardo Mendes, 20240850
- Helena Duarte, 20240530
- João Freire, 20240528
- Mariana Sousa, 20240516

<div class="alert alert-block alert-info">

# Table of Contents
    
[1. Import Libraries](#1)<br>

[2. Load data](#2)<br>

<a class="anchor" id="1">

# 1. Import Libraries
    
</a>

In [30]:
import os
import pandas as pd

from copy import deepcopy
from random import random, sample, choice, randint
import copy

from abc import ABC, abstractmethod

<a class="anchor" id="2">

# 2. Load data
    
</a>

In [2]:
#data_dir= os.path.join(os.getcwd(), 'players(in).csv')

df = pd.read_csv("players(in).csv", index_col=0)
df.head()

,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95


* create a list with all records of the players as a dict

In [3]:
players = df.to_dict(orient="records")
players

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95},
 {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
 {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
 {'Name': 'Mason Reed', 'Position': 'DEF', 'Skill': 82, 'Salary (€M)': 75},
 {'Name': 'Logan Brooks', 'Position': 'DEF', 'Skill': 86, 'Salary (€M)': 95},
 {'Name': 'Caleb Fisher', 'Position': 'DEF', 'Skill': 84, 'Salary (€M)': 85},
 {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Sa

<a class="anchor" id="3">

# 3. Problem Definiton
    
</a>

In a fantasy sports league, the objective is to assign players to teams in a way that ensures
a balanced distribution of talent while staying within salary caps.

1) Each player is defined by the following attributes:
* Skill rating: Represents the player's ability.
* Cost: The player's salary.
* Position (One of four roles) : Goalkeeper (GK), Defender (DEF), Midfielder (MID), or Forward (FWD).

A solution is a complete league configuration, specifying the team assignment for each player. These are the constraints that must be verified in every solution of the search space (no object is considered a solution if it doesn’t comply with these):
* Each team must consist of: 1 Goalkeeper, 2 Defenders, 2 Midfielders and 2
Forwards.
* Each player is assigned to exactly one team.

*Impossible Configurations*: Teams that do not follow this exact structure (e.g., a team with 2 goalkeepers, or a team where the same defender is assigned twice) are not part of the search space and are not considered solutions. It is forbidden to generate such an arrangement during evolution.

Besides that, each team should not exceed a 750€ million total budget. If it does, it is not a valid solution and the fitness value should reflect that.

The `objective` is to create a balanced league that complies with the constraints. 
A balanced league a is a league where the average skill rating of the players is roughly the same among the teams. 
This can be measured by the standard deviation of the average skill rating of the teams.

You can find a dataset of players with their names, position, skill rating and salary (in million €).
These players should be distributed across 5 teams of 7 players each.

# 4.Representação com classes player, team and League 

In [6]:
players # list of players, each player is a dictionary

[{'Name': 'Alex Carter', 'Position': 'GK', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Jordan Smith', 'Position': 'GK', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ryan Mitchell', 'Position': 'GK', 'Skill': 83, 'Salary (€M)': 85},
 {'Name': 'Chris Thompson', 'Position': 'GK', 'Skill': 80, 'Salary (€M)': 80},
 {'Name': 'Blake Henderson', 'Position': 'GK', 'Skill': 87, 'Salary (€M)': 95},
 {'Name': 'Daniel Foster', 'Position': 'DEF', 'Skill': 90, 'Salary (€M)': 110},
 {'Name': 'Lucas Bennett', 'Position': 'DEF', 'Skill': 85, 'Salary (€M)': 90},
 {'Name': 'Owen Parker', 'Position': 'DEF', 'Skill': 88, 'Salary (€M)': 100},
 {'Name': 'Ethan Howard', 'Position': 'DEF', 'Skill': 80, 'Salary (€M)': 70},
 {'Name': 'Mason Reed', 'Position': 'DEF', 'Skill': 82, 'Salary (€M)': 75},
 {'Name': 'Logan Brooks', 'Position': 'DEF', 'Skill': 86, 'Salary (€M)': 95},
 {'Name': 'Caleb Fisher', 'Position': 'DEF', 'Skill': 84, 'Salary (€M)': 85},
 {'Name': 'Nathan Wright', 'Position': 'MID', 'Skill': 92, 'Sa

In [7]:
df

,Name,Position,Skill,Salary (€M)
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95
5,Daniel Foster,DEF,90,110
6,Lucas Bennett,DEF,85,90
7,Owen Parker,DEF,88,100
8,Ethan Howard,DEF,80,70
9,Mason Reed,DEF,82,75


In [8]:
# change the column names to lowercase
df.rename(columns={
    "Name": "name",
    "Position": "position",
    "Skill": "skill",
    "Salary (€M)": "salary"
}, inplace=True)

In [9]:
df

,name,position,skill,salary
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95
5,Daniel Foster,DEF,90,110
6,Lucas Bennett,DEF,85,90
7,Owen Parker,DEF,88,100
8,Ethan Howard,DEF,80,70
9,Mason Reed,DEF,82,75


In [10]:
position_order = ["GK", "DEF", "MID", "FWD"] # the order of positions

# Create a mapping dictionary: {'GK': 0, 'DEF': 1, 'MID': 2, 'FWD': 3}
# x.map(...) converts each "position" value in the DataFrame to its corresponding order index
# Sorts the DataFrame according to these mapped indices
df_sorted = df.sort_values(by="position", key=lambda x: x.map({pos: i for i, pos in enumerate(position_order)})).reset_index(drop=True)
df_sorted

,name,position,skill,salary
0,Alex Carter,GK,85,90
1,Jordan Smith,GK,88,100
2,Ryan Mitchell,GK,83,85
3,Chris Thompson,GK,80,80
4,Blake Henderson,GK,87,95
5,Maxwell Flores,DEF,81,72
6,Jaxon Griffin,DEF,79,65
7,Caleb Fisher,DEF,84,85
8,Logan Brooks,DEF,86,95
9,Ethan Howard,DEF,80,70


In [11]:
df_sorted["id"]= df_sorted.index

In [12]:
df_sorted


,name,position,skill,salary,id
0,Alex Carter,GK,85,90,0
1,Jordan Smith,GK,88,100,1
2,Ryan Mitchell,GK,83,85,2
3,Chris Thompson,GK,80,80,3
4,Blake Henderson,GK,87,95,4
5,Maxwell Flores,DEF,81,72,5
6,Jaxon Griffin,DEF,79,65,6
7,Caleb Fisher,DEF,84,85,7
8,Logan Brooks,DEF,86,95,8
9,Ethan Howard,DEF,80,70,9


In [13]:
players = df_sorted.to_dict(orient="records")
players

[{'name': 'Alex Carter', 'position': 'GK', 'skill': 85, 'salary': 90, 'id': 0},
 {'name': 'Jordan Smith',
  'position': 'GK',
  'skill': 88,
  'salary': 100,
  'id': 1},
 {'name': 'Ryan Mitchell',
  'position': 'GK',
  'skill': 83,
  'salary': 85,
  'id': 2},
 {'name': 'Chris Thompson',
  'position': 'GK',
  'skill': 80,
  'salary': 80,
  'id': 3},
 {'name': 'Blake Henderson',
  'position': 'GK',
  'skill': 87,
  'salary': 95,
  'id': 4},
 {'name': 'Maxwell Flores',
  'position': 'DEF',
  'skill': 81,
  'salary': 72,
  'id': 5},
 {'name': 'Jaxon Griffin',
  'position': 'DEF',
  'skill': 79,
  'salary': 65,
  'id': 6},
 {'name': 'Caleb Fisher',
  'position': 'DEF',
  'skill': 84,
  'salary': 85,
  'id': 7},
 {'name': 'Logan Brooks',
  'position': 'DEF',
  'skill': 86,
  'salary': 95,
  'id': 8},
 {'name': 'Ethan Howard',
  'position': 'DEF',
  'skill': 80,
  'salary': 70,
  'id': 9},
 {'name': 'Owen Parker',
  'position': 'DEF',
  'skill': 88,
  'salary': 100,
  'id': 10},
 {'name': 'Lu

In [14]:
team_order= ["GK", "DEF", "MID", "FWD"]
team_grouped = df_sorted.groupby("position")["id"].apply(list)
position_id_map = {pos: team_grouped[pos] for pos in team_order if pos in team_grouped}

In [15]:
position_id_map

{'GK': [0, 1, 2, 3, 4],
 'DEF': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'MID': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 'FWD': [25, 26, 27, 28, 29, 30, 31, 32, 33, 34]}

In [16]:
import random


In [17]:
def generate_team_indices(players,position_id_map):
    # generates a team from a dictionary of indices

    # Randomly select one valid team (just player indices)
    team = []
    team += random.sample(position_id_map["GK"], 1)  # Select 1 goalkeeper
    team += random.sample(position_id_map["DEF"], 2) # Select 2 defenders
    team += random.sample(position_id_map["MID"], 2) # Select 2 midfielders
    team += random.sample(position_id_map["FWD"], 2) # Select 2 forwards

    return team

In [18]:
team_indices = generate_team_indices(players,position_id_map)
team_indices

[1, 6, 12, 22, 16, 26, 34]

In [19]:
team_indices = generate_team_indices(players,position_id_map)  

print("Team (indices):", team_indices)
print("Team (players):")
for i in team_indices:
    p = players[i]
    print(f"- {p['name']} ({p['position']}) Skill: {p['skill']}, Salary: €{p['salary']}M")

Team (indices): [2, 6, 13, 17, 18, 34, 30]
Team (players):
- Ryan Mitchell (GK) Skill: 83, Salary: €85M
- Jaxon Griffin (DEF) Skill: 79, Salary: €65M
- Mason Reed (DEF) Skill: 82, Salary: €75M
- Dominic Bell (MID) Skill: 86, Salary: €95M
- Gavin Richardson (MID) Skill: 87, Salary: €95M
- Zachary Nelson (FWD) Skill: 86, Salary: €92M
- Chase Murphy (FWD) Skill: 86, Salary: €95M


In [20]:
def generate_league(players):
    from copy import deepcopy
    import random
    num_teams=5
    
    # Step 1: Copy the player pool
    available_indices = list(range(len(players)))
    random.shuffle(available_indices)  # randomize pool to start
    
    # Step 2: Group indices by position
    def group_available(indices):
        from collections import defaultdict
        pos_map = defaultdict(list)
        for i in indices:
            pos_map[players[i]["position"]].append(i)
        return pos_map
    
    league = []

    for _ in range(num_teams):
        pos_to_indices = group_available(available_indices)

        # Check we still have enough players per role
        if (len(pos_to_indices["GK"]) < 1 or
            len(pos_to_indices["DEF"]) < 2 or
            len(pos_to_indices["MID"]) < 2 or
            len(pos_to_indices["FWD"]) < 2):
            raise ValueError("Not enough players left to form a complete team.")

        team = []
        team += random.sample(pos_to_indices["GK"], 1)
        team += random.sample(pos_to_indices["DEF"], 2)
        team += random.sample(pos_to_indices["MID"], 2)
        team += random.sample(pos_to_indices["FWD"], 2)

        # Remove these players from the available pool
        for idx in team:
            available_indices.remove(idx)

        league.append(team)

    return league


In [21]:
league = generate_league(players)  # this gives you a league

# Just display the structure (no formatting, no printing names)
league

[[2, 8, 14, 23, 22, 32, 33],
 [1, 5, 12, 18, 19, 30, 29],
 [4, 13, 10, 21, 20, 27, 34],
 [0, 7, 6, 16, 24, 28, 31],
 [3, 11, 9, 17, 15, 26, 25]]

# SOLUTION REPRESENTATION

In [22]:
from abc import ABC, abstractmethod

In [23]:
class Solution(ABC):
    def __init__(self, repr=None):
        # To initialize a solution we need to know it's representation.
        # If no representation is given, a representation is randomly initialized.
        if repr == None:
            repr = self.random_initial_representation()
        # Attributes
        self.repr = repr

    # Method that is called when we run print(object of the class)
    def __repr__(self):
        return str(self.repr)

    # Other methods that must be implemented in subclasses
    @abstractmethod
    def fitness(self):
        pass

    @abstractmethod
    def random_initial_representation():
        pass


In [24]:
df_sorted.head(5)

,name,position,skill,salary,id
0,Alex Carter,GK,85,90,0
1,Jordan Smith,GK,88,100,1
2,Ryan Mitchell,GK,83,85,2
3,Chris Thompson,GK,80,80,3
4,Blake Henderson,GK,87,95,4


In [25]:
def generate_league(df):

    import random
    from collections import defaultdict
    
    num_teams=5
    available_ids = df.index.tolist()
    random.shuffle(available_ids)

    league = []

    for _ in range(num_teams):
        pos_map = defaultdict(list)

        # Build position map using current available players
        for i in available_ids:
            pos = df.loc[i, "position"]
            pos_map[pos].append(i)

        # Check we have enough players left per role
        if (len(pos_map["GK"]) < 1 or
            len(pos_map["DEF"]) < 2 or
            len(pos_map["MID"]) < 2 or
            len(pos_map["FWD"]) < 2):
            raise ValueError("Not enough players left to form a full team")

        # Select players for the team
        team = []
        team += random.sample(pos_map["GK"], 1)
        team += random.sample(pos_map["DEF"], 2)
        team += random.sample(pos_map["MID"], 2)
        team += random.sample(pos_map["FWD"], 2)

        # Remove them from pool
        for idx in team:
            available_ids.remove(idx)

        league.append(team)

    return league

In [26]:
class SportsLeagueSolution(Solution):
    def __init__(self, repr=None, players_df=df_sorted):
        self.players_df = players_df
        super().__init__(repr=repr)

    def random_initial_representation(self):
        self.repr = generate_league(self.players_df)
        return self.repr

    def fitness(self):
        current_league = self.repr  # list of team index lists
        team_skills = []
        team_salaries = []

        for current_league in league:
            team_df = self.players_df.loc[team]
            total_skill = team_df["skill"].sum()
            total_salary = team_df["salary"].sum()

            # Budget constraint
            if total_salary > 750:
                return 1e9  

            team_skills.append(total_skill)
            team_salaries.append(total_salary)

        avg_skill = sum(team_skills) / len(team_skills)
        salary_std = pd.Series(team_salaries).std()

        # Maximize skill, penalize unbalanced salary
        return avg_skill - 0.1 * salary_std


In [27]:
sol1 = SportsLeagueSolution()
sol1

[[0, 10, 7, 18, 16, 26, 28], [2, 11, 8, 23, 15, 29, 33], [1, 13, 5, 19, 20, 32, 31], [4, 9, 6, 21, 24, 30, 34], [3, 14, 12, 22, 17, 25, 27]]

In [28]:
sol2 = SportsLeagueSolution()
sol2

[[4, 14, 12, 19, 16, 27, 30], [3, 10, 7, 18, 23, 34, 33], [1, 11, 9, 20, 15, 32, 29], [2, 6, 13, 21, 24, 31, 26], [0, 8, 5, 17, 22, 28, 25]]

In [29]:
position_id_map

{'GK': [0, 1, 2, 3, 4],
 'DEF': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'MID': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 'FWD': [25, 26, 27, 28, 29, 30, 31, 32, 33, 34]}

* sugestão de crossover (exemplo): podem, por exemplo, manter todos os jogadores de uma determinada posição do pai1 no filho1, 
nas mesmas equipas em que se encontram no pai1. Depois, para outra posição, podem usar a distribuição desses jogadores pelas equipas do pai2, e assim sucessivamente.

* A vossa sugestão de “trocar jogadores entre equipas dentro da mesma liga” é considerado uma mutação e não um crossover.

# Mutation

Mutation types:
1. PlayerSwap: Basic switch one player with another in the same position
2. RoleShuffle: Choose a role, remove all players from that role, shuffle and re-atribute to teams
3. PlayerRoleLeftShift: Select a player role (GK, DEF, MID, FWD) and shifts the players in that role to the left across teams, by a random number of positions

In [85]:
from abc import ABC, abstractmethod
from random import randint, shuffle, choice
from copy import deepcopy

In [31]:
class MutationOperator(ABC):
    @abstractmethod
    def mutate(self, solution):
        pass

In [ ]:
class PlayerSwapMutation(MutationOperator):
    def mutate(self, solution, verbose=False):
        new_repr = deepcopy(solution.repr)

        # Choose the id within the team of the player that will be swapped
        player_to_swap = randint(0, 6)

        # Choose the teams where the players will be swapped. Make sure they are different
        team_to_swap_1 = randint(0, 4)
        team_to_swap_2 = randint(0, 4)
        while team_to_swap_1 == team_to_swap_2:
            team_to_swap_2 = randint(0, 4)

        if verbose:
            # Extract player IDs before swap for accurate logging
            pid1 = new_repr[team_to_swap_1][player_to_swap]
            pid2 = new_repr[team_to_swap_2][player_to_swap]

            print(f"Swapping player {pid1} from team {team_to_swap_1} "
                f"with player {pid2} from team {team_to_swap_2}")

        # Swap players at the chosen index
        new_repr[team_to_swap_1][player_to_swap], new_repr[team_to_swap_2][player_to_swap] = pid2, pid1

        mutated = deepcopy(solution)
        mutated.repr = new_repr
        return mutated


In [51]:
original_solution = SportsLeagueSolution(players_df=df_sorted)
original_repr = deepcopy(original_solution.repr)  # Capture pre-mutation state

mutation = PlayerSwapMutation()
mutated_solution = mutation.mutate(original_solution, verbose=True)

print(f"Original Solution: {original_repr}")
print(f"Mutated Solution: {mutated_solution}")


Swapping player 13 from team 2 with player 10 from team 4
Original Solution: [[4, 14, 5, 19, 20, 25, 29], [2, 9, 7, 18, 16, 30, 31], [1, 13, 6, 24, 21, 34, 27], [0, 12, 11, 23, 17, 33, 26], [3, 10, 8, 22, 15, 28, 32]]
Mutated Solution: [[4, 14, 5, 19, 20, 25, 29], [2, 9, 7, 18, 16, 30, 31], [1, 10, 6, 24, 21, 34, 27], [0, 12, 11, 23, 17, 33, 26], [3, 13, 8, 22, 15, 28, 32]]


In [ ]:
class RoleShuffleMutation(MutationOperator):
    def mutate(self, solution, verbose=False):
        new_repr = deepcopy(solution.repr)
        
        # Choose the role that will be affected
        # Remembering that the player IDs withing the team correspond to {"GK": 0, "DEF": [1, 2], "MID": [3, 4], "FWD": [5, 6]}
        i = randint(0, 6)
        if i in [1, 2]:
            i = [1, 2]
        elif i in [3, 4]:
            i = [3, 4]
        elif i in [5, 6]:
            i = [5, 6]
        else:
            i = [0]
        
        # If verbose, print the role and indexes being shuffled
        if verbose:
            role_map = {
                "GK": [0],
                "DEF": [1, 2],
                "MID": [3, 4],
                "FWD": [5, 6]
            }
            inv_map = {tuple(v): k for k, v in role_map.items()}
            role_name = inv_map[tuple(i)]
            print(f"Shuffling players in role {role_name}, corresponding to indexes {i}")


        # Remove all the players from the selected role and shuffle them
        bag_of_players = []
        for team in new_repr:
            bag_of_players += [team[i] for i in i]
            
        shuffle(bag_of_players)

        # Once shuffled, put them back in the teams
        index = 0
        for team in new_repr:
            for j in i:
                team[j] = bag_of_players[index]
                index += 1

        mutated = deepcopy(solution)
        mutated.repr = new_repr
        return mutated


In [ ]:
original_solution = SportsLeagueSolution(players_df=df_sorted)
original_repr = deepcopy(original_solution.repr)  # Capture pre-mutation state

mutation = RoleShuffleMutation()
mutated_solution = mutation.mutate(original_solution, verbose=True)

print(f"Original Solution: {original_repr}")
print(f"Mutated Solution: {mutated_solution}")


Shuffling players in position MID, corresponding to indexes [3, 4]
Original Solution: [[0, 8, 14, 18, 15, 31, 33], [1, 13, 12, 21, 19, 32, 26], [2, 5, 7, 17, 24, 29, 28], [3, 9, 11, 20, 23, 27, 34], [4, 6, 10, 16, 22, 30, 25]]
Mutated Solution: [[0, 8, 14, 19, 24, 31, 33], [1, 13, 12, 16, 23, 32, 26], [2, 5, 7, 17, 18, 29, 28], [3, 9, 11, 20, 22, 27, 34], [4, 6, 10, 15, 21, 30, 25]]


In [98]:
class PlayerRoleLeftShiftMutation(MutationOperator):
    def mutate(self, solution, verbose=False):
        new_repr = deepcopy(solution.repr)

        # Choose the role that will be affected
        # Remembering that the player IDs withing the team correspond to {"GK": 0, "DEF": [1, 2], "MID": [3, 4], "FWD": [5, 6]}
        i = randint(0, 6)
        if i in [1, 2]:
            i = [1, 2]
        elif i in [3, 4]:
            i = [3, 4]
        elif i in [5, 6]:
            i = [5, 6]
        else:
            i = [0]
        
        # Get all the players from the selected role
        role_players = []
        for team in new_repr:
            for idx in i:
                role_players.append(team[idx])

        # Shift left 
        shift_amount = randint(1, len(new_repr) - 1)
        role_players = role_players[shift_amount:] + role_players[:shift_amount]
        
        # If verbose, print the role and indexes being shifted
        if verbose:
            role_map = {
                "GK": [0],
                "DEF": [1, 2],
                "MID": [3, 4],
                "FWD": [5, 6]
            }
            inv_map = {tuple(v): k for k, v in role_map.items()}
            role_name = inv_map[tuple(i)]
            print(f"Shifting role group {role_name}, corresponding to indexes {i}, by {shift_amount} positions")

        # Reassign to teams
        index = 0
        for team in new_repr:
            for idx in i:
                team[idx] = role_players[index]
                index += 1


        mutated = deepcopy(solution)
        mutated.repr = new_repr
        return mutated


In [104]:
original_solution = SportsLeagueSolution(players_df=df_sorted)
original_repr = deepcopy(original_solution.repr)  # Capture pre-mutation state

mutation = PlayerRoleLeftShiftMutation()
mutated_solution = mutation.mutate(original_solution, verbose=True)

print(f"Original Solution: {original_repr}")
print(f"Mutated Solution: {mutated_solution}")


Shifting role group FWD, corresponding to indexes [5, 6], by 1 positions
Original Solution: [[0, 8, 13, 20, 19, 25, 32], [4, 12, 10, 16, 23, 28, 33], [2, 11, 14, 24, 21, 29, 26], [3, 9, 7, 15, 17, 31, 34], [1, 6, 5, 18, 22, 27, 30]]
Mutated Solution: [[0, 8, 13, 20, 19, 32, 28], [4, 12, 10, 16, 23, 33, 29], [2, 11, 14, 24, 21, 26, 31], [3, 9, 7, 15, 17, 34, 27], [1, 6, 5, 18, 22, 30, 25]]


In [ ]:
# class TeamRecombinationMutation(MutationOperator):
#     def mutate(self, solution, verbose=False):
#         new_repr = deepcopy(solution.repr)
        
#         # Choose the teams that will be recombined
#         team_to_recombine_1 = randint(0, 4)
#         team_to_recombine_2 = randint(0, 4)
#         while team_to_recombine_1 == team_to_recombine_2:
#             team_to_recombine_2 = randint(0, 4)
            
#         # Choose the breakpoint for the recombination
#         breakpoint = randint(1, 5)
        
#         # If verbose, print the breakpoint and teams being recombined
#         if verbose:
#             print(f"Recombining teams {team_to_recombine_1} and {team_to_recombine_2} at breakpoint {breakpoint}")

#         # Recombine the teams
#         team1 = new_repr[team_to_recombine_1]
#         team2 = new_repr[team_to_recombine_2]

#         new_team1 = team1[:breakpoint] + team2[breakpoint:]
#         new_team2 = team2[:breakpoint] + team1[breakpoint:]

#         new_repr[team_to_recombine_1] = new_team1
#         new_repr[team_to_recombine_2] = new_team2

#         mutated = deepcopy(solution)
#         mutated.repr = new_repr
#         return mutated


In [84]:
# original_solution = SportsLeagueSolution(players_df=df_sorted)
# original_repr = deepcopy(original_solution.repr)  # Capture pre-mutation state

# mutation = TeamRecombinationMutation()
# mutated_solution = mutation.mutate(original_solution, verbose=True)

# print(f"Original Solution: {original_repr}")
# print(f"Mutated Solution: {mutated_solution}")


Recombining teams 2 and 1 at breakpoint 3
Original Solution: [[2, 7, 5, 15, 22, 27, 33], [4, 12, 14, 24, 17, 31, 25], [0, 9, 13, 16, 18, 34, 26], [3, 8, 10, 20, 21, 30, 29], [1, 6, 11, 23, 19, 28, 32]]
Mutated Solution: [[2, 7, 5, 15, 22, 27, 33], [4, 12, 14, 16, 18, 34, 26], [0, 9, 13, 24, 17, 31, 25], [3, 8, 10, 20, 21, 30, 29], [1, 6, 11, 23, 19, 28, 32]]
